In [1]:
import os

In [2]:
pwd

'/Users/nilakshanraveendran/Documents/Text-Summarization-by-NLP/research'

In [4]:
os.chdir("../")

In [5]:
pwd

'/Users/nilakshanraveendran/Documents/Text-Summarization-by-NLP'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path 

In [24]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [25]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config

In [26]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [27]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(
                f"{filename}, File downloaded successfully! with following info: \n{headers} and the size of file is {get_size(Path(filename))}"
            )

        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        Extracts the zip file to the unzip directory
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"File extracted successfully at: {unzip_path}")

In [28]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()

    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2025-11-17 20:23:34,414]: INFO: common: YAML file: config/config.yaml loaded successfully
[2025-11-17 20:23:34,422]: INFO: common: YAML file: params.yaml loaded successfully
[2025-11-17 20:23:34,423]: INFO: common: Directory created at: artifacts
[2025-11-17 20:23:34,424]: INFO: common: Directory created at: artifacts/data_ingestion
[2025-11-17 20:23:38,856]: INFO: 2878176782: artifacts/data_ingestion/samsumdata.zip, File downloaded successfully! with following info: 
Connection: close
Content-Length: 23627009
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "2e7409b328d118a1d37018be788babf8bf9640386387da766ac100e96efa3b93"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 4E9D:C52BD:4AA81D:505D18:691B36EF
Accept-Ranges: bytes
Date: Mon, 17 Nov 2025 14:53:37 GMT
Via: 1.1 varnish
X-Served-By: c